In [14]:
using Sockets, Serialization, Random, JSON3, Dates

# Конфигурация по умолчанию
const DEFAULT_POPULATION_SIZE = 100
const DEFAULT_MUTATION_RATE = 0.01
const DEFAULT_CROSSOVER_RATE = 0.7
const DEFAULT_MAX_GENERATIONS = 100
const DEFAULT_MIGRATION_INTERVAL = 10 # Интервал миграции в поколениях
const DEFAULT_NUMBER_OF_WORKERS = 2
const DEFAULT_ISOLATION_TIME = 5 # Default isolation time

# Функция Розенброка
function rosenbrock(x, y)
    a = 1.0
    b = 100.0
    return (a - x)^2 + b * (y - x^2)^2
end

# Структура для представления индивида
mutable struct IndividualA
    genes::Vector{Float64}
    fitness::Float64
end

# Функция для инициализации популяции
function initialize_population(population_size::Int, gene_length::Int, min_val::Float64, max_val::Float64)
    population = Vector{IndividualA}(undef, population_size)
    for i in 1:population_size
        genes = [rand() * (max_val - min_val) + min_val for _ in 1:gene_length]
        population[i] = IndividualA(genes, 0.0)
    end
    return population
end

# Функция для оценки приспособленности индивида (функция Розенброка)
function evaluate_fitness!(IndividualA::IndividualA)
    IndividualA.fitness = -rosenbrock(IndividualA.genes[1], IndividualA.genes[2]) # Минимизация, поэтому берем отрицательное значение
end

# Функция для выполнения селекции (турнирный отбор)
function selection(population::Vector{IndividualA}, tournament_size::Int)
    selected = Vector{IndividualA}(undef, length(population))
    for i in 1:length(population)
        tournament = rand(1:length(population), tournament_size)
        winner = tournament[argmax([population[j].fitness for j in tournament])]
        selected[i] = IndividualA(copy(population[winner].genes), population[winner].fitness) # Важно копировать гены
    end
    return selected
end

# Функция для выполнения кроссовера (одноточечный)
function crossover!(parent1::IndividualA, parent2::IndividualA, crossover_rate::Float64)
    if rand() < crossover_rate
        point = rand(1:length(parent1.genes)-1)
        for i in point:length(parent1.genes)
            parent1.genes[i], parent2.genes[i] = parent2.genes[i], parent1.genes[i]
        end
    end
end

# Функция для выполнения мутации (с адаптивной вероятностью мутации)
function mutate!(IndividualA::IndividualA, mutation_rate::Float64, gene_length::Int)
    # Вычисляем среднее значение генов
    gene_mean = sum(IndividualA.genes) / gene_length

    # Вычисляем стандартное отклонение генов
    gene_std = std(IndividualA.genes)

    # Адаптивная вероятность мутации для каждого гена
    for i in 1:length(IndividualA.genes)
        # Вычисляем близость гена к среднему
        gene_proximity = abs(IndividualA.genes[i] - gene_mean) / gene_std

        # Адаптируем вероятность мутации на основе близости к среднему
        adapted_mutation_rate = mutation_rate * (1 + gene_proximity) # Чем дальше от среднего, тем выше вероятность мутации

        # Применяем мутацию с адаптивной вероятностью
        if rand() < adapted_mutation_rate
            IndividualA.genes[i] += randn() * 0.1 # Нормальное распределение для мутации
        end
    end
end

# Функция для выполнения генетического алгоритма
function genetic_algorithm(population_size::Int, gene_length::Int, min_val::Float64, max_val::Float64,
                           mutation_rate::Float64, crossover_rate::Float64, max_generations::Int)
    population = initialize_population(population_size, gene_length, min_val, max_val)

    for generation in 1:max_generations
        # Оценка приспособленности
        for IndividualA in population
            evaluate_fitness!(IndividualA)
        end

        # Сортировка популяции по приспособленности (от лучшего к худшему)
        sort!(population, by = x -> x.fitness, rev = true)

        # Вывод лучшего индивида в каждом поколении
        println("Generation: ", generation, ", Best Fitness: ", population[1].fitness, ", Genes: ", population[1].genes)

        # Селекция
        selected_population = selection(population, 5) # tournament size = 5

        # Кроссовер
        for i in 1:2:length(selected_population)-1
            crossover!(selected_population[i], selected_population[i+1], crossover_rate)
        end

        # Мутация
        for IndividualA in selected_population
            mutate!(IndividualA, mutation_rate, gene_length)
        end

        # Обновление популяции
        population = selected_population
    end

    # Оценка приспособленности финальной популяции
    for IndividualA in population
        evaluate_fitness!(IndividualA)
    end

    # Сортировка финальной популяции
    sort!(population, by = x -> x.fitness, rev = true)

    return population
end

# Функция для выполнения генетического алгоритма с миграцией
function migrating_genetic_algorithm(population_size::Int, gene_length::Int, min_val::Float64, max_val::Float64,
                                     mutation_rate::Float64, crossover_rate::Float64, max_generations::Int,
                                     migration_interval::Int, num_workers::Int, isolation_time::Int)

    populations = [initialize_population(population_size, gene_length, min_val, max_val) for _ in 1:num_workers]
    best_IndividualAs = Vector{IndividualA}(undef, num_workers)  # Для хранения лучших особей с каждого острова

    for generation in 1:max_generations
        for worker_id in 1:num_workers
            population = populations[worker_id]

            # Оценка приспособленности
            for IndividualA in population
                evaluate_fitness!(IndividualA)
            end

            # Сортировка популяции по приспособленности (от лучшего к худшему)
            sort!(population, by = x -> x.fitness, rev = true)

            # Сохраняем лучшую особь для миграции
            best_IndividualAs[worker_id] = deepcopy(population[1])

            # Селекция
            selected_population = selection(population, 5)

            # Кроссовер
            for i in 1:2:length(selected_population)-1
                crossover!(selected_population[i], selected_population[i+1], crossover_rate)
            end

            # Мутация
            for IndividualA in selected_population
                mutate!(IndividualA, mutation_rate, gene_length)
            end

            populations[worker_id] = selected_population
        end

        # Миграция
        if generation % migration_interval == 0
            # Выбираем случайных особей для замены
            for worker_id in 1:num_workers
                num_migrants = div(population_size, num_workers)  # Количество мигрантов с каждого острова
                immigrant_indices = rand(1:population_size, num_migrants)

                # Отправляем лучших особей на другие острова
                for i in 1:num_workers
                    target_worker = (worker_id + i) % num_workers + 1
                    populations[target_worker][immigrant_indices[i]] = deepcopy(best_IndividualAs[worker_id])
                end
            end

            println("Migration occurred at generation ", generation)
        end

        # Вывод лучшего индивида в каждом поколении
        best_fitness = maximum([populations[i][1].fitness for i in 1:num_workers])
        best_genes = populations[argmax([populations[i][1].fitness for i in 1:num_workers])][1].genes
        println("Generation: ", generation, ", Best Fitness: ", best_fitness, ", Genes: ", best_genes)
    end

    # Оценка финальных популяций
    for worker_id in 1:num_workers
        population = populations[worker_id]
        for IndividualA in population
            evaluate_fitness!(IndividualA)
        end
        sort!(population, by = x -> x.fitness, rev = true)
        populations[worker_id] = population
    end

    return populations
end

function main()
    # Параметры генетического алгоритма
    population_size = 100
    gene_length = 2  # Для функции Розенброка нужно 2 параметра (x, y)
    min_val = -5.0
    max_val = 5.0
    mutation_rate = 0.01
    crossover_rate = 0.7
    max_generations = 100

    # Параметры для миграции
    migration_interval = 10
    num_workers = 4
    isolation_time = 5

    # Запуск классического генетического алгоритма
    println("Starting Classic Genetic Algorithm...")
    start_time = now()
    final_population_classic = genetic_algorithm(population_size, gene_length, min_val, max_val, mutation_rate, crossover_rate, max_generations)
    end_time = now()
    elapsed_time_classic = end_time - start_time
    println("Classic Genetic Algorithm completed in: ", elapsed_time_classic)
    println("Best IndividualA (Classic): ", final_population_classic[1])

    # Запуск генетического алгоритма с миграцией
    println("\nStarting Migrating Genetic Algorithm...")
    start_time = now()
    final_populations_migrating = migrating_genetic_algorithm(population_size, gene_length, min_val, max_val, mutation_rate, crossover_rate, max_generations, migration_interval, num_workers, isolation_time)
    end_time = now()
    elapsed_time_migrating = end_time - start_time
    println("Migrating Genetic Algorithm completed in: ", elapsed_time_migrating)

    # Вывод лучшего индивида из всех популяций после миграции
    best_IndividualA_migrating = final_populations_migrating[argmax([populations[1].fitness for populations in final_populations_migrating])][1]
    println("Best IndividualA (Migrating): ", best_IndividualA_migrating)

    # Сравнение результатов
    println("\nComparison:")
    println("Classic GA - Best Fitness: ", final_population_classic[1].fitness)
    println("Migrating GA - Best Fitness: ", best_IndividualA_migrating.fitness)

    # Пример качественного анализа (Можно добавить более детальный анализ)
    if final_population_classic[1].fitness > best_IndividualA_migrating.fitness
        println("Classic GA found a better solution.")
    else
        println("Migrating GA found a better solution.")
    end

end

main()

Starting Classic Genetic Algorithm...
Generation: 1, Best Fitness: -0.7988339716133579, Genes: [0.2143461928336965, 0.08855673773028183]
Generation: 2, Best Fitness: -0.6480210436734739, Genes: [0.2457947319687298, 0.08855673773028183]
Generation: 3, Best Fitness: -0.6480210436734739, Genes: [0.2457947319687298, 0.08855673773028183]
Generation: 4, Best Fitness: -0.6480210436734739, Genes: [0.2457947319687298, 0.08855673773028183]
Generation: 5, Best Fitness: -0.514843480315895, Genes: [0.34245566509505293, 0.08855673773028183]
Generation: 6, Best Fitness: -0.514843480315895, Genes: [0.34245566509505293, 0.08855673773028183]
Generation: 7, Best Fitness: -0.514843480315895, Genes: [0.34245566509505293, 0.08855673773028183]
Generation: 8, Best Fitness: -0.47101112210283047, Genes: [0.34245566509505293, 0.13693461337363988]
Generation: 9, Best Fitness: -0.47101112210283047, Genes: [0.34245566509505293, 0.13693461337363988]
Generation: 10, Best Fitness: -0.4340356391356807, Genes: [0.342455